In [44]:
import sys
import os
import re
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple


In [46]:
import pandas as pd
import numpy as np

IGNORED_FILES = ['.DS_Store']
RAW_DATA_ROOT_PATH = 'data/kaggle_raw/'



In [82]:
# Writes a list of paths to files in data directory
def write_data_paths():
    data_paths = []
    for subdir, dirs, files in os.walk(RAW_DATA_ROOT_PATH):
        for file in files:
            if file not in IGNORED_FILES:
                data_paths.append(os.path.join(subdir, file))
    return data_paths



data_paths = write_data_paths()
WIC_dir_re = re.compile('data\/kaggle_raw\/WICAgencies*.{4}ytd\/.*\.csv')
WIC_data_paths = [path for path in data_paths if re.findall(WIC_dir_re, path)]

subtitle = re.findall(re.compile('data\/kaggle_raw\/WICAgencies*.{4}ytd\/(.*)\.csv'),WIC_data_paths[0])[0]
subtitle
headers = [" ".join([subtitle,col.split()[0]]) for col in list(pd.read_csv(WIC_data_paths[0], nrows=1))]
headers
#WIC_df = pd.read_csv(WIC_data_paths[0], index_col='State Agency or Indian Tribal Organization',usecols =[i for i in headers if i != 'Cumulative Average'], names=headers, header=0)
#
#WIC_df
#wic_agencies_13_ytd = pd.read_csv("data/kaggle_raw/WICAgencies2013ytd")
#wic_agencies_13_ytd

['Infants_Partially_Breastfed State',
 'Infants_Partially_Breastfed 2014-10-01',
 'Infants_Partially_Breastfed 2014-11-01',
 'Infants_Partially_Breastfed 2014-12-01',
 'Infants_Partially_Breastfed 2015-01-01',
 'Infants_Partially_Breastfed 2015-02-01',
 'Infants_Partially_Breastfed 2015-03-01',
 'Infants_Partially_Breastfed 2015-04-01',
 'Infants_Partially_Breastfed 2015-05-01',
 'Infants_Partially_Breastfed 2015-06-01',
 'Infants_Partially_Breastfed 2015-07-01',
 'Infants_Partially_Breastfed 2015-08-01',
 'Infants_Partially_Breastfed 2015-09-01',
 'Infants_Partially_Breastfed Average']